![PyData_logo](./static/pydata-logo-madrid-2016.png)

# Remove Before Flight
## Analyzing Flight Safety Data with Python

###### Jesús Martos Carrizo
###### Alejandro Sáez Mollejo

### Introduction

#### Some figures 

![passengers](./static/passenger.jpg)
###### by Ryan McGuire

####  Flight Safety

#### Obejctive of this talk 

##### Disclaimer

#### Some remarks about

![pandas-logo](./static/pandas_logo.png)

#### Data source

<img src="./static/ntsb.jpg" style="width: 150px;"/>

__National Transportation Safety Board__ is charged with determining the probable cause of transportation accidents and promoting transportation safety, and assisting victims of transportation accidents and their families.

Data: http://www.ntsb.gov/_layouts/ntsb.aviation/index.aspx

### Loading & Cleaning data 

In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Some configuration options:
pd.set_option('max_columns', 70)
pd.set_option('max_rows', 50)

# to show all posiible options:
# pd.describe_option()

# to reset to default option value:
# pd.reset_option('max_columns')

# to show current value:
# pd.get_option('max_columns')

In [3]:
events = pd.read_csv("./data/events.csv",
                                    sep='|')  # Use eventid as index was generating problems
occurrences = pd.read_csv("./data/Occurrences1.csv",
                                    sep=';',
                                    index_col=0)  # Use eventid as index

/miniconda/envs/pydata-safety/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 31 events with NaN date
print(events['ev_date'].isnull().sum())

# 345 events with NaN time
print(events['ev_time'].isnull().sum())

31
345


##### A first glipse of the data 

In [5]:
events.head(5)
# there is an extra row for ev_id with no data because we have used index_col=0 as an argument in the function read_csv()

,ev_id,ntsb_no,ev_type,ev_date,ev_dow,ev_time,ev_tmzn,ev_city,ev_state,ev_country,ev_site_zipcode,ev_year,ev_month,mid_air,on_ground_collision,latitude,longitude,latlong_acq,apt_name,ev_nr_apt_id,ev_nr_apt_loc,apt_dist,apt_dir,apt_elev,wx_brief_comp,wx_src_iic,wx_obs_time,wx_obs_dir,wx_obs_fac_id,wx_obs_elev,wx_obs_dist,wx_obs_tmzn,light_cond,sky_cond_nonceil,sky_nonceil_ht,...,sky_cond_ceil,vis_rvr,vis_rvv,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_dir_ind,wind_vel_kts,wind_vel_ind,gust_ind,gust_kts,altimeter,wx_dens_alt,wx_int_precip,metar,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,invest_agy,ntsb_docket,ntsb_notf_from,ntsb_notf_date,ntsb_notf_tm,fiche_number,lchg_date,lchg_userid,wx_cond_basic,faa_dist_office
0,20001208X07734,LAX97FA143,ACC,04/02/1997 12:00:00 AM,WE,2016.0,MST,GRAND CANYON,AZ,USA,86023,1997.0,4,N,N,,,NaN,,,OFAP,NaN,NaN,NaN,PARP,WFAC,1954.0,180.0,GCN,6606.0,21.0,MST,NDRK,SCAT,3600.0,...,BKN,"0,00000000e+00",0.0,"1,00000000e+01",-2.0,-3.0,0.0,U,0.0,CALM,N,0.0,"2,90000000e+01",NaN,UNK,NaN,FATL,0.0,0.0,0.0,2.0,NaN,NaN,NaN,2.0,N,1890.0,NaN,04/02/1997 12:00:00 AM,1922.0,IMAGE,12/08/2000 12:13:19 PM,dbo,IMC,NaN
1,20021008X05297,CHI02LA299,ACC,09/29/2002 12:00:00 AM,Su,2100.0,CDT,Moorhead,MN,USA,56560,2002.0,9,N,N,465021N,0963947W,NaN,Moorhead Municipal,JKJ,ONAP,NaN,NaN,917.0,UNK,WFAC,2053.0,47.0,FAR,908.0,9.0,BST,NDRK,CLER,NaN,...,OVC,NaN,NaN,"7,00000000e+00",14.0,12.0,160.0,Y,13.0,SPEC,Y,0.0,"2,97099991e+01",NaN,NaN,NaN,SERS,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,N,13246.0,FAA,10/03/2002 12:00:00 AM,945.0,DMS,04/23/2003 12:26:44 PM,NTSB\\MONR,VMC,Minneapolis FSDO
2,20040127X00111,LAX04LA103,ACC,10/12/2003 12:00:00 AM,Su,1640.0,MST,Scottsdale,AZ,USA,85255,2003.0,10,N,N,333722N,1115438W,NaN,Scottsdale,SDL,ONAP,NaN,NaN,1510.0,FULL,WFAC,1653.0,NaN,KSDL,1510.0,NaN,MST,DAYL,CLER,NaN,...,NONE,NaN,NaN,"1,00000000e+01",33.0,12.0,NaN,NaN,NaN,CALM,N,NaN,"2,98899994e+01",NaN,NaN,NaN,NONE,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,N,17586.0,FAA,01/22/2004 12:00:00 AM,1215.0,DMS,04/28/2005 02:59:26 PM,NTSB\\JOHB,VMC,WP-SDL-FSDO
3,20001213X34369,ANC86LA135,ACC,08/23/1986 12:00:00 AM,SA,1100.0,AKD,TALKEETNA,AK,USA,99676,1986.0,8,N,N,,,NaN,,,OFAP,"0,00000000e+00",0.0,0.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,CLER,0.0,...,NONE,"0,00000000e+00",0.0,"5,00000000e+01",10.0,-18.0,0.0,U,0.0,UNK,U,0.0,NaN,NaN,UNK,NaN,NONE,0.0,0.0,0.0,NaN,NaN,3.0,NaN,NaN,N,891.0,NaN,08/23/1986 12:00:00 AM,1430.0,31238,01/02/2001 09:14:50 AM,dbo,VMC,NaN
4,20001213X35290,ANC87LA019,ACC,12/07/1986 12:00:00 AM,SU,1256.0,AST,SHAGELUK,AK,USA,99665,1986.0,12,N,N,,,NaN,,1,OFAP,"0,00000000e+00",0.0,0.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,SCAT,1500.0,...,NONE,"0,00000000e+00",0.0,"4,00000000e+01",-1.0,-1.0,0.0,U,0.0,CALM,N,0.0,"2,90000000e+01",NaN,LGT,NaN,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,N,1400.0,NaN,12/07/1986 12:00:00 AM,1300.0,31791,01/02/2001 09:14:55 AM,dbo,VMC,NaN


In [6]:
events.columns

Index(['ev_id', 'ntsb_no', 'ev_type', 'ev_date', 'ev_dow', 'ev_time',
       'ev_tmzn', 'ev_city', 'ev_state', 'ev_country', 'ev_site_zipcode',
       'ev_year', 'ev_month', 'mid_air', 'on_ground_collision', 'latitude',
       'longitude', 'latlong_acq', 'apt_name', 'ev_nr_apt_id', 'ev_nr_apt_loc',
       'apt_dist', 'apt_dir', 'apt_elev', 'wx_brief_comp', 'wx_src_iic',
       'wx_obs_time', 'wx_obs_dir', 'wx_obs_fac_id', 'wx_obs_elev',
       'wx_obs_dist', 'wx_obs_tmzn', 'light_cond', 'sky_cond_nonceil',
       'sky_nonceil_ht', 'sky_ceil_ht', 'sky_cond_ceil', 'vis_rvr', 'vis_rvv',
       'vis_sm', 'wx_temp', 'wx_dew_pt', 'wind_dir_deg', 'wind_dir_ind',
       'wind_vel_kts', 'wind_vel_ind', 'gust_ind', 'gust_kts', 'altimeter',
       'wx_dens_alt', 'wx_int_precip', 'metar', 'ev_highest_injury',
       'inj_f_grnd', 'inj_m_grnd', 'inj_s_grnd', 'inj_tot_f', 'inj_tot_m',
       'inj_tot_n', 'inj_tot_s', 'inj_tot_t', 'invest_agy', 'ntsb_docket',
       'ntsb_notf_from', 'ntsb_notf_date'

In [7]:
# Why should we change the column names?
# Moreover, names with capitals and spaces make it more complicated for us
# to select the column (ie. `events.ev_dow` is valid and `events.Event day of the week` is not)

#events = events.rename(columns = {'ev_dow':'Event day of the week','light_cond':'Light Conditions','wind_vel_kts':'Wind Speed(knots)',
#                       'metar':'METAR Weather Report','vis_rvr':'Visibility Runway Visual Range (Feet)',
#                        'vis_rvv':'Visibility Runway Visual Value (Statute Miles)','vis_sm':'Visibility (Statute Miles)',
#                        'wx_cond_basic':'Basic Weather Conditions'})

In [8]:
events.dtypes

ev_id                   object
ntsb_no                 object
ev_type                 object
ev_date                 object
ev_dow                  object
ev_time                float64
ev_tmzn                 object
ev_city                 object
ev_state                object
ev_country              object
ev_site_zipcode         object
ev_year                float64
ev_month                object
mid_air                 object
on_ground_collision     object
latitude                object
longitude               object
latlong_acq             object
apt_name                object
ev_nr_apt_id            object
ev_nr_apt_loc           object
apt_dist                object
apt_dir                float64
apt_elev               float64
wx_brief_comp           object
                        ...   
gust_ind                object
gust_kts               float64
altimeter               object
wx_dens_alt            float64
wx_int_precip           object
metar                   object
ev_highe

In [9]:
# Not really useful for the majority of columns
events.describe()

,ev_time,ev_year,apt_dir,apt_elev,wx_obs_time,wx_obs_dir,wx_obs_elev,wx_obs_dist,sky_nonceil_ht,sky_ceil_ht,vis_rvv,wx_temp,wx_dew_pt,wind_dir_deg,wind_vel_kts,gust_kts,wx_dens_alt,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,ntsb_docket,ntsb_notf_tm
count,76535.000000,76848.000000,37020.000000,48146.000000,69909.000000,63990.000000,67552.000000,65299.000000,56454.000000,55382.000000,48754.000000,67043.000000,55773.000000,67986.000000,69080.000000,53729.000000,20314.000000,49522.000000,49526.000000,49513.000000,15253.000000,14347.000000,46273.000000,10793.000000,35217.000000,72695.000000,56995.000000
mean,1373.671196,1996.920427,49.827769,1055.010344,999.464661,103.528911,866.237728,11.339745,3255.918819,2930.506861,0.146039,19.850603,6.877970,161.884167,7.257426,3.650245,1772.910357,0.010157,0.013064,0.011209,2.813545,1.866732,7.574676,1.543037,2.451969,8586.409781,1043.181577
std,434.386422,101.770110,100.047186,1655.995540,720.507141,122.243560,3746.138486,30.473635,6264.894555,6349.779335,22.881972,17.341558,15.660181,117.043219,5.951389,10.485619,2706.629533,0.353946,0.393807,0.518030,11.025730,5.172820,31.982420,2.722736,8.733082,10805.425263,718.420703
min,0.000000,1948.000000,0.000000,-210.000000,0.000000,0.000000,-115.000000,0.000000,0.000000,0.000000,0.000000,-62.000000,-38.000000,0.000000,0.000000,0.000000,-30536.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1100.000000,1988.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,45.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,945.000000,1.000000
50%,1400.000000,1996.000000,0.000000,434.000000,1153.000000,40.000000,128.000000,1.000000,0.000000,0.000000,0.000000,21.000000,8.000000,170.000000,7.000000,0.000000,500.000000,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2005.000000,1200.000000
75%,1700.000000,2005.000000,20.000000,1071.000000,1553.000000,200.000000,845.000000,16.000000,4000.000000,2700.000000,0.000000,27.000000,16.000000,260.000000,10.000000,0.000000,2370.750000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,16468.500000,1600.000000
max,2400.000000,26733.000000,979.000000,13250.000000,2400.000000,360.000000,751635.000000,1526.000000,250000.000000,250000.000000,4444.000000,1652.000000,1646.000000,999.000000,210.000000,999.000000,26858.000000,44.000000,62.000000,71.000000,349.000000,380.000000,699.000000,111.000000,380.000000,34935.000000,2400.000000


In [10]:
# Lets gather some info about the data
events.info(max_cols=0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76880 entries, 0 to 76879
Columns: 71 entries, ev_id to faa_dist_office
dtypes: float64(27), object(44)
memory usage: 41.6+ MB


In [11]:
# What does 41.6+ MB mean???
events.info(max_cols=0, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76880 entries, 0 to 76879
Columns: 71 entries, ev_id to faa_dist_office
dtypes: float64(27), object(44)
memory usage: 209.2 MB


In [12]:
events.memory_usage(deep=True).sum() / 1048  # KiB  

209329.88167938931

In [13]:
# with deep=False same as events.values.nbyte; what is the difference when is True?
events.values.nbytes / 1048

41667.78625954199

In [14]:
events.info(max_cols=0, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76880 entries, 0 to 76879
Columns: 71 entries, ev_id to faa_dist_office
dtypes: float64(27), object(44)
memory usage: 209.2 MB


##### Selecting only certain events 

##### Accidents 

In [15]:
# Maybe there is a better way to select...
# Make a brief comparison of results and performance (%timeit)
events = events[events['ev_type'] == 'ACC']
# options:
# events = events.where(events['ev_dow']=='WE')       # the return a DataFrame of the same shape as the original.When the logical 
                                                    # condition in False the result is the whole row filled with NaNs.
                                                    # By default, where returns a modified copy of the data. There is an 
                                                    # optional parameter inplace so that the original data can be modified
                                                    # without creating a copy

# events = events.where(events['ev_dow']=='WE', 0, inplace=True)     # same that above but choosing the result instead of NaN and 
                                                                 # without creating a copy

# events = events.mask(events['ev_dow']=='WE')      # mask is the inverse boolean operation of where

# events = events.query('ev_type == ACC')     #  query() method that allows selection using an expression. In pandas 0.18 is 
                                              # an experimental method. Can this become useful?

##### Kind of flight 

In [16]:
most_victims = events['inj_tot_t'].sort_values(ascending=False).iloc[0:150]
most_victims.head(10)

11753    349.0
64434    295.0
2842     283.0
67843    275.0
31855    265.0
15820    256.0
54086    254.0
38759    239.0
45452    231.0
41787    230.0
Name: inj_tot_t, dtype: float64

In [17]:
most_victims.count()

150

In [18]:
aircraft = pd.read_csv("./data/aircraft.csv",
                                    encoding = "ISO-8859-1",  # export the table again.
                                    sep=';')

# check DtypteWarning 
# Columns (4,7,17,27,28,32,37,38,47,48,53,63,70,72,73,76,82) have mixed types.
# Specify dtype option on import or set low_memory=False.

/miniconda/envs/pydata-safety/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4,7,17,27,28,32,37,38,47,48,53,63,70,72,73,76,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
aircrafts_most_victims = aircraft[aircraft['ev_id'].isin(most_victims.index)]
aircrafts_most_victims

,ev_id,Aircraft_Key,regis_no,ntsb_no,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,acft_series,acft_serial_no,cert_max_gr_wt,acft_category,acft_reg_cls,homebuilt,fc_seats,cc_seats,pax_seats,total_seats,num_eng,fixed_retractable,type_last_insp,date_last_insp,afm_hrs_last_insp,afm_hrs,elt_install,elt_oper,elt_aided_loc_ev,elt_type,owner_acft,owner_street,...,second_pilot,dprt_pt_same_ev,dprt_apt_id,dprt_city,dprt_state,dprt_country,dprt_time,dprt_timezn,dest_same_local,dest_apt_id,dest_city,dest_state,dest_country,phase_flt_spec,report_to_icao,evacuation,lchg_date,lchg_userid,afm_hrs_since,rwy_num,rwy_len,rwy_width,site_seeing,air_medical,med_type_flight,acft_year,fuel_on_board,commercial_space_flight,unmanned,ifr_equipped_cert,elt_mounted_aircraft,elt_connected_antenna,elt_manufacturer,elt_model,elt_reason_other


In [20]:
# Number of accidents in each far part
aircrafts_most_victims['far_part'].value_counts()

Series([], Name: far_part, dtype: int64)

In [21]:
# Aircraft make
aircrafts_most_victims['acft_make'].value_counts() 

Series([], Name: acft_make, dtype: int64)

In [22]:
# Some of them are in caps
aircrafts_most_victims['acft_make'] = aircrafts_most_victims['acft_make'].str.lower()
aircrafts_most_victims['acft_make'].value_counts() 

Series([], Name: acft_make, dtype: int64)

In [23]:
# Far part
# Check again!
desired_far_parts = ['NUSC',  # Non-U.S. Commercial
                     '121',      # Air Carrier
                     '129',      # Foreign
                   # '091',      # General Aviation
                     'NUSN',  # Non-U.S. Non-Commercial
                     'UNK',     # Unknown
                     '135']      # Air Taxi & Commuter

In [24]:
cond = aircraft['far_part'].isin(desired_far_parts)
ev_ids_for_desired_far = aircraft[cond]['ev_id'].drop_duplicates()

ev_ids_for_desired_far.count()

ev_ids_for_desired_far.values

array(['20001204X00000', '20001204X00001', '20001204X00002', ...,
       '20160210X70158', '20160212X11826', '20160222X93713'], dtype=object)

In [25]:
# Using ev_id as index is behaving different here
#events = events.loc[ev_ids_for_desired_far.values]
#events['ev_type'].count()

In [26]:
events = events[events['ev_id'].isin(ev_ids_for_desired_far.values)]
events

,ev_id,ntsb_no,ev_type,ev_date,ev_dow,ev_time,ev_tmzn,ev_city,ev_state,ev_country,ev_site_zipcode,ev_year,ev_month,mid_air,on_ground_collision,latitude,longitude,latlong_acq,apt_name,ev_nr_apt_id,ev_nr_apt_loc,apt_dist,apt_dir,apt_elev,wx_brief_comp,wx_src_iic,wx_obs_time,wx_obs_dir,wx_obs_fac_id,wx_obs_elev,wx_obs_dist,wx_obs_tmzn,light_cond,sky_cond_nonceil,sky_nonceil_ht,...,sky_cond_ceil,vis_rvr,vis_rvv,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_dir_ind,wind_vel_kts,wind_vel_ind,gust_ind,gust_kts,altimeter,wx_dens_alt,wx_int_precip,metar,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,invest_agy,ntsb_docket,ntsb_notf_from,ntsb_notf_date,ntsb_notf_tm,fiche_number,lchg_date,lchg_userid,wx_cond_basic,faa_dist_office
40,20140507X51410,DCA14CA090,ACC,03/04/2014 12:00:00 AM,Tu,2300.0,PST,San Diego,CA,USA,92101,2014.0,3,N,N,324406N,1171123W,NaN,San Diego International,SAN,ONAP,"0,00000000e+00",NaN,17.0,NaN,UNK,651.0,NaN,SAN,NaN,NaN,UTC,DAYL,FEW,NaN,...,NONE,NaN,NaN,"1,00000000e+01",15.0,11.0,20.0,Y,3.0,SPEC,N,NaN,"3,00799999e+01",NaN,NaN,NaN,SERS,NaN,NaN,NaN,NaN,NaN,186.0,1.0,1.0,N,NaN,Delta Air Lines Flight Safety,NaN,NaN,NaN,09/10/2015 04:49:58 PM,lovj,VMC,NaN
46,20001213X30836,ANC87FA067,ACC,05/07/1987 12:00:00 AM,TH,1415.0,AKD,NIGHTMUTE,AK,USA,99690,1987.0,5,N,N,,,NaN,,,OFAP,"0,00000000e+00",0.0,NaN,UNK,WIT,0.0,0.0,,0.0,0.0,,DAYL,CLER,0.0,...,OBSC,"0,00000000e+00",0.0,"1,00000000e+00",-1.0,-2.0,140.0,U,6.0,UNK,N,0.0,"2,90000000e+01",NaN,LGT,NaN,FATL,0.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0,N,217.0,NaN,05/09/1987 12:00:00 AM,2200.0,33175,12/13/2000 02:22:27 PM,dbo,IMC,NaN
50,20001213X26058,NYC88LA152,ACC,06/01/1988 12:00:00 AM,WE,850.0,EDT,UTICA,NY,USA,13502,1988.0,6,N,N,,,NaN,,,OFAP,"0,00000000e+00",0.0,NaN,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,UNK,0.0,...,OVC,"0,00000000e+00",0.0,"3,00000000e+00",21.0,NaN,290.0,U,5.0,UNK,U,0.0,NaN,NaN,LGT,NaN,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,N,1955.0,NaN,06/01/1988 12:00:00 AM,1115.0,38223,12/13/2000 11:51:24 AM,dbo,VMC,NaN
56,20050825X01307,DCA05RA088,ACC,08/02/2005 12:00:00 AM,Tu,1600.0,EDT,Toronto,NaN,CA,NaN,2005.0,8,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NONE,NaN,NaN,NaN,NaN,NaN,309.0,NaN,NaN,O,NaN,TSB,NaN,NaN,NaN,11/04/2005 02:15:07 PM,NTSB\\JOHB,NaN,NaN
63,20151123X43041,ERA16WA047,ACC,11/17/2015 12:00:00 AM,Tu,1550.0,UTC,Choco,FN,CO,NaN,2015.0,11,N,N,083020N,0771624W,EST,Acandi,SKAD,OFAP,NaN,NaN,50.0,NaN,UNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FATL,NaN,NaN,NaN,2.0,NaN,NaN,8.0,10.0,O,NaN,Government of Colombia,NaN,NaN,NaN,11/30/2015 12:04:29 PM,kenj,VMC,NaN
92,20041013X01621,DCA04WA083,ACC,08/30/2004 12:00:00 AM,Mo,1.0,EST,Sydney,NaN,AS,NaN,2004.0,8,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NONE,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,O,NaN,ATSB,NaN,NaN,NaN,10/13/2004 04:08:28 PM,HAUT,NaN,NaN
111,20140619X32751,WPR14LA251,ACC,06/17/2014 12:00:00 AM,Tu,1330.0,HST,Wailuku,HI,USA,96793,2014.0,6,N,N,205355N,1562550W,MEAS,NaN,NaN,OFAP,NaN,NaN,NaN,NaN,UNK,954.0,NaN,OGG,NaN,3.0,HST,DAYL,CLER,NaN,...,BKN,NaN,NaN,"2,00000000e+01",28.0,3.0,45.0,Y,18.0,SPEC,Y,31.0,"3,00200005e+01",NaN,NaN,NaN,NONE,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,N,NaN,FAA WPR ROC,NaN,NaN,NaN,07/02/2014 12:53:03 PM,stam,VMC,NaN
116,20001213X32847,DEN86LA079,ACC,02/18/1986 12:00:00 AM,TU,920.0,MST,GRAND JUNCTION,CO,USA,81500,1986.0,2,N,N,,,NaN,,,OFAP,"0,00000000e+00",0.0,0.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,UNK,0.0,...,BKN,"0,00000000e+00",0.0,"2,00000000e+01",-1.0,NaN,180.0,U,20.0,UNK,U,40.0,NaN,10572.0,UNK,NaN,MINR,0.0,0.0,0.0,NaN,3.0,NaN,NaN,3.0,N,663.0,NaN,02/18/1986 12:00:00 AM,1315.0,30949,01/02/2001 09:14:40 AM,dbo,VMC,NaN
120,20001214X41483,SEA85LA010,ACC,10/15/1984 12:00:00 AM,MO,1051.0,PDT,JUNEAU,AK,USA,

##### Converting types 

##### date & time 

In [27]:
# 0 events with NaN date
print(events['ev_date'].isnull().sum())

# 115 events with NaN time
print(events['ev_time'].isnull().sum())

0
115


In [28]:
# Information about date format: 
# https://docs.python.org/3.5/library/datetime.html#strftime-and-strptime-behavior
date_format = "%m/%d/%Y %I:%M:%S %p"
% timeit pd.to_datetime(events['ev_date'], format=date_format)

10 loops, best of 3: 67.2 ms per loop


In [29]:
date_format = "%m/%d/%Y %I:%M:%S %p"
date = pd.to_datetime(events['ev_date'], format=date_format)

print(date.isnull().sum())

0


In [30]:
time = events['ev_time'].astype(str).str.split('.').str[0].str.zfill(4)

# There are 2 wrong values in time: 2400 and 2391
# correponding to index: 27642 and 64448
# they will transform to 00:00 h
time.replace(['0nan', '2400', '2391'], '0000', inplace=True)
time = pd.to_datetime(time,
                      format="%H%M",  # format
                      errors='coerce')  # for nan values

print(time.isnull().sum())

0


In [31]:
# Sample
str_date = date.dt.date.astype(str)
str_time = time.dt.time.astype(str)

str_date.iloc[0] + ' ' + str_time.iloc[0]

'2014-03-04 23:00:00'

In [32]:
date_time = pd.to_datetime(str_date + ' ' + str_time,
                           format="%Y-%m-%d %H:%M:%S")
date_time.isnull().sum()

0

In [33]:
events['ev_date'] = date_time

In [34]:
# ev_dow can be discarded since that info is contained in the datetime format:
events['ev_date'].dt.dayofweek.head()

40    1
46    3
50    2
56    1
63    1
Name: ev_date, dtype: int64

In [35]:
events.drop('ev_dow', axis=1, inplace=True)

##### Latitude & longitude 

In [36]:
print(events['latitude'].isnull().sum())
print(events['longitude'].isnull().sum())

1628
1627


In [37]:
def convert_lat(string):
    degs = float(string[0:2])
    mins = float(string[2:4])
    secs = float(string[4:6])
    last = string[6].lower()
    if last == 's':
        factor = -1.0
    elif last == 'n':
        factor = 1.0
    else:
        raise ValueError("invalid hemisphere")
    return factor * (degs + mins / 60 + secs / 3600)

In [38]:
print('NaN:' ,events['latitude'].isnull().sum())
print('Total:', events['latitude'].count())

lat = events['latitude'].dropna()
mask = lat.str.contains(r'^[0-9]{6}[NnSs]$')
events['latitude_num'] = lat[mask].apply(convert_lat)
events['latitude_num'].head()
# pd.notnull(events['latitude_num']).count() is the same with 6 and {5,6} in regex

print('Matching Pattern: ', mask.sum())
print('Not matching: ', events['latitude'].count() - mask.sum())

NaN: 1628
Total: 4830
Matching Pattern:  2151
Not matching:  2679


In [39]:
# Values not matching... DO NOT obey the rules!
print(lat[~mask])

46              
50              
116             
120             
129             
257             
435             
442             
499             
524             
530             
540             
686             
760             
769             
899             
963             
965             
995             
1129            
1141            
1176            
1190            
1232            
1237            
          ...   
76227           
76247           
76256           
76285           
76287           
76309           
76341           
76362           
76377           
76383           
76407           
76424           
76493           
76501           
76503           
76509           
76540           
76596           
76614           
76688           
76701           
76712           
76750           
76836           
76850           
Name: latitude, dtype: object


In [40]:
def convert_lon(string):
    degs = float(string[0:3])
    mins = float(string[3:5])
    secs = float(string[5:7])
    last = string[7].lower()
    if last == 'w':
        factor = -1.0
    elif last == 'e':
        factor = 1.0
    else:
        raise ValueError("invalid direction")
    return factor * (degs + mins / 60 + secs / 3600)

In [41]:
print('NaN:' ,events['longitude'].isnull().sum())
print('Total:', events['longitude'].count())

lon = events['longitude'].dropna()
mask = lon.str.contains(r'^[0-9]{7}[EeWw]$')
events['longitude_num'] = lon[mask].apply(convert_lon)
events['longitude_num'].head()
# pd.notnull(events['longitude_num']).count() is the same with 7 and {6,7} in regex
# and the same as notnull in latitude. Yuhu!

print('Matching Pattern: ', mask.sum())
print('Not matching: ', events['longitude'].count() - mask.sum())

NaN: 1627
Total: 4831
Matching Pattern:  2149
Not matching:  2682


In [42]:
# Values not matching... DO NOT obey the rules!
lon[~mask]

46               
50               
116              
120              
129              
257              
435              
442              
499              
524              
530              
540              
686              
760              
769              
899              
963              
965              
995              
1129             
1141             
1176             
1190             
1232             
1237             
           ...   
76227            
76247            
76256            
76285            
76287            
76309            
76341            
76362            
76377            
76383            
76407            
76424            
76493            
76501            
76503            
76509            
76540            
76596            
76614            
76688            
76701            
76712            
76750            
76836            
76850            
Name: longitude, dtype: object

In [43]:
#events.drop(['latitude', 'longitude', 'latlong_acq'], axis=1, inplace=True)

In [44]:
%matplotlib qt
# miller projection
from mpl_toolkits.basemap import Basemap
map = Basemap('mill',lon_0=0, lat_0=0)
# plot coastlines, draw label meridians and parallels.
map.drawcoastlines()
# fill continents 'coral' (with zorder=0), color wet areas 'aqua'
#map.drawmapboundary(fill_color='aqua')
#map.fillcontinents(color='coral',lake_color='aqua')
map.bluemarble()
map.scatter(events['longitude_num'].values, events['latitude_num'].values, latlon=True, marker='o', color='r')

### Data Analysis 

#### Accidents location 

#### Accident trends 

#### When do accidents occur? 

In [45]:
a = occurrences['Phase_of_Flight'].value_counts()
b = a[0:7]  #  Why are we discarding the rest of maneuvers?

maneuvers = {572:'LANDING-ROLL',
             571:'LANDING - FLARE/TOUCHDOWN',
             553:'DESCENT - UNCONTROLLED',
             552:'DESCENT - EMERGENCY (plt initiated)',
             522:'TAKEOFF - INITIAL CLIMB',
             580:'MANEUVERING (includes buzzing)',
             540:'CRUISE (includes low altitude straight and level flight)'}

b = b.rename_axis(maneuvers)

b

LANDING-ROLL                                                21339
LANDING - FLARE/TOUCHDOWN                                   14871
DESCENT - UNCONTROLLED                                      13874
DESCENT - EMERGENCY (plt initiated)                         10714
TAKEOFF - INITIAL CLIMB                                      9746
MANEUVERING (includes buzzing)                               6607
CRUISE (includes low altitude straight and level flight)     6300
Name: Phase_of_Flight, dtype: int64

#### Other Questions... 

### Conclusions 

# Thanks for yor attention! 

![PyData_logo](./static/pydata-logo-madrid-2016.png)

## Any Questions?


---


In [46]:
# Notebook style
from IPython.core.display import HTML
css_file = './static/style.css'
HTML(open(css_file, "r").read())